## Получаем данные

In [6]:
import pandas as pd

In [7]:
import numpy as np
from tqdm import tqdm_notebook
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import KFold

In [74]:
df_ratings = pd.read_csv('./ml-1m/ratings.dat', sep='::', header=None)
df_movies = pd.read_csv('./ml-1m/movies.dat', sep='::', header=None)

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [86]:
df_ratings.columns=['userId','movieId','rating','timestamp']
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [87]:
df_movies.columns=['movieId','title','genres']
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [88]:
df = pd.merge(df_ratings, df_movies, on='movieId')

In [89]:
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


In [91]:
df_for_surpise = df_ratings[['userId', 'movieId', 'rating']]

In [92]:
df_for_surpise.columns = ['uid', 'iid', 'rating']

In [93]:
df_for_surpise.head()

,uid,iid,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [94]:
reader = Reader(rating_scale=(0.5, 5))

In [95]:
dataset = Dataset.load_from_df(df_for_surpise, reader)

In [96]:
trainset, testset = train_test_split(dataset, test_size=0.2)

In [148]:
algo = KNNBasic(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})

In [149]:
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [150]:
predictions = algo.test(testset)

In [151]:
accuracy.rmse(predictions)

RMSE: 0.9247


0.9246761007607519

In [139]:
kfold = KFold(5)

In [155]:
scores = []
for trainset, testset in tqdm_notebook(kfold.split(dataset)):
    algo = KNNBasic(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})
    algo.fit(trainset)
    predictions = algo.test(testset)
    scores.append(accuracy.rmse(predictions))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9261
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9275
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9279
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9266
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9261


In [156]:
scores

[0.9260989238124766,
 0.927519127977361,
 0.9278681221695743,
 0.9266263756171328,
 0.9260547918655502]